In [ ]:
!pip install -q transformers torch gradio pandas

import torch
import pandas as pd
import gradio as gr
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Device Setup
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load GPT-2 (for fake news generation)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(Device)

# Load BERT (for fake news detection)
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2
).to(Device)

# Feedback log (stored in memory)
feedback_log = []

# === Fake News Generator ===
def generate_fake_news(prompt):
    inputs = gpt2_tokenizer.encode(prompt, return_tensors="pt").to(Device)
    outputs = gpt2_model.generate(
        inputs,
        max_length=200,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        early_stopping=True
    )
    return gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)

# === Fake News Detector ===
def detect_news(text):
    if not isinstance(text, str) or not text.strip():
        return "Please enter some text.", 0.0

    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(Device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    logits = outputs.logits

    if logits.shape[0] == 1:
        logits = logits.squeeze(0)

    predicted_class = torch.argmax(logits, dim=-1).item()
    confidence = torch.softmax(logits, dim=-1)[predicted_class].item()

    # If confidence is low, treat as Fake News
    if confidence < 0.75:
        label = "Fake News ⚠️ (Low Confidence – Treated as Fake)"
    else:
        label = "Fake News" if predicted_class == 0 else "Real News"

    return f"{label} (Confidence: {confidence:.2f})", confidence

# === Word Counter ===
def count_words(text):
    return f"📄 Word Count: {len(text.split())}"

# === Feedback Recorder ===
def record_feedback(article_text, result, feedback):
    feedback_log.append({
        "Text": article_text,
        "Result": result,
        "Feedback": feedback
    })
    return f"Feedback received: {feedback} 👍" if feedback == "Agree" else f"Feedback received: {feedback} 👎"

# === Dark Theme ===
dark_theme = gr.themes.Base(
    primary_hue="blue",
    neutral_hue="slate"
).set(
    body_background_fill="#111827",
    body_text_color="#e5e7eb",
    input_background_fill="#1f2937",
    input_border_color="#374151",
    block_background_fill="#1e293b"
)

# === Gradio Interface ===
with gr.Blocks(theme=dark_theme) as demo:
    gr.Markdown("## 🧠 Fake News Generator & Detector (Confidence-Aware)")

    with gr.Tab("🧪 Generate Fake News"):
        input_text = gr.Textbox(label="Enter a Headline or Prompt", lines=2)
        generate_btn = gr.Button("Generate")
        clear_btn = gr.Button("Clear")
        output_text = gr.Textbox(label="Generated News", lines=15)

        generate_btn.click(generate_fake_news, inputs=input_text, outputs=output_text)
        clear_btn.click(lambda: "", outputs=input_text)

    with gr.Tab("🛡 Detect Fake or Real"):
        detect_input = gr.Textbox(label="Paste News Article or Statement", lines=6)
        detect_btn = gr.Button("Detect")
        word_count = gr.Textbox(label="📄 Word Count", interactive=False)
        detect_output = gr.Textbox(label="Detection Result")
        confidence_bar = gr.Slider(minimum=0, maximum=1, label="Confidence", interactive=False)

        feedback_msg = gr.Textbox(label="📢 Feedback Message", interactive=False)
        agree_btn = gr.Button("👍 Agree")
        disagree_btn = gr.Button("👎 Disagree")

        detect_input.change(count_words, inputs=detect_input, outputs=word_count)
        detect_btn.click(detect_news, inputs=detect_input, outputs=[detect_output, confidence_bar])
        agree_btn.click(record_feedback, inputs=[detect_input, detect_output, gr.Textbox(value="Agree", visible=False)], outputs=feedback_msg)
        disagree_btn.click(record_feedback, inputs=[detect_input, detect_output, gr.Textbox(value="Disagree", visible=False)], outputs=feedback_msg)

# Launch the app
demo.launch()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b6f75f9955ca25642e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
